# Фильтрация текстов для претрейна LLM

**Тестовое задание DS 2026 — Точка банк**

## Описание задачи

Необходимо отфильтровать «грязный» датасет для претрейна LLM. Тексты оцениваются по трём критериям (бинарная классификация 0/1):

| Критерий | Описание | 1 | 0 |
|----------|----------|---|---|
| **Integrity** (единство темы) | Можно ли озаглавить текст и выделить главную мысль? | Да (статьи) | Нет (списки, каталоги) |
| **Factuality** (фактологичность) | Содержит ли текст факты о мире? | Да (статьи, описания) | Нет (мнения, перечисления) |
| **Truthfulness** (правдивость) | Не противоречит ли текст common-sense знаниям? | Да | Нет (фантастика, вымысел) |

**Метрика:** среднее арифметическое F1 по каждому критерию. Примеры с меткой 0.5 исключаются из расчёта.

## Подход и гипотезы

### Какие гипотезы проверялись и что дали

1. **Rule-based baseline** — эвристики по длине строк, пунктуации, доле «списочных» паттернов. Даёт быстрый ориентир и неплохо ловит очевидный мусор (каталоги, списки городов), но не различает семантически сложные случаи. Использовался как sanity check.

2. **ML на числовых признаках (sklearn, XGBoost)** — длина текста, количество предложений, доля стоп-слов, пунктуации и т.п. Baseline выше rule-based, но упирается в потолок: фичи не отражают смысл текста. Подходит как быстрый фильтр, но для задачи фильтрации претрейн-данных этого мало.

3. **Отдельные BERT-модели на каждый критерий** — три модели вместо одной. Оказалось хуже, чем multi-task: каждая модель видит меньше данных и не использует общие представления. Отдельные «головы» на общем энкодер-представлении дали лучшие результаты.

4. **Multi-Task BERT** — одна модель, три классификационные головы. Общий энкодер учится представлениям, полезным для всех трёх критериев. Лучше работает на ограниченных данных и даёт более стабильный средний F1.

5. **Разморозка только последних слоёв** — при полном fine-tune BERT быстро переобучался. Обучение только последних 2 слоёв + pooler + головы дало лучший компромисс между выразительностью и устойчивостью.

---

### Почему нельзя оптимизировать F1 для класса 1

Класс **1** (качественный текст) — доминирующий класс в выборке. Если оптимизировать F1 по классу 1, модель может достичь высокого скор, почти всегда предсказывая 1: много истинно положительных, мало ложно отрицательных. Фактически это тривиальная стратегия «предсказывать мажорный класс», и она не решает задачу фильтрации мусора.

Класс **0** (мусор, низкое качество) — редкий, но именно его важно находить: в претрейне попадание плохих данных ухудшает модель. Поэтому метрика строится по F1 класса 0: важно и не пропускать мусор (recall), и не выбрасывать лишние хорошие тексты (precision). В loss закладываем именно F1 для класса 0.

---

### Precision vs Recall: бизнес-выбор

Торг между precision и recall для класса 0 определяет, как мы фильтруем данные:

- **Высокий recall, ниже precision**  
  Фильтруем агрессивнее: выкидываем больше «сомнительных», в т.ч. часть хороших. Меньше плохих данных попадает в претрейн, но потери качественных текстов выше.

- **Высокий precision, ниже recall**  
  Фильтруем консервативнее: удаляем только то, в чём уверены. Больше хороших данных сохраняется, но часть мусора проходит в датасет.

Выбор зависит от цели: минимизировать «загрязнение» модели (выше recall) или сохранить максимум полезных данных (выше precision). В loss можно менять веса precision/recall (например, через β в Fβ) под конкретные требования продукта.

---

### Дальнейшие направления при наличии ресурсов

При большем бюджете (GPU, время) имеет смысл:

- **Крупные модели** (например, rubert-large, XLM-R large) — лучше улавливают семантику и границы между классами, дают потенциальный прирост скор.
- **Более долгое обучение** с чекпоинтами и early stopping — можно лучше подобрать число эпох и снизить риск переобучения.
- **Ансамбли** (несколько BERT + разморозка разных слоёв) — обычно дают более стабильный и высокий средний F1.

---

> 📁 **P.S.** Все эти эксперименты (и не только) есть в изначальном ноутбуке `Untitled1_(1).ipynb`. Предупреждаю: там ~187 ячеек, оформление на уровне «запустил и побежал» + говнокод, что я сам через месяц в нём не разберутся. Но зато каждая гипотеза — в наличии. 👀

## 1. Импорты и настройка

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer
from datasets import Dataset
from tqdm import tqdm

In [ ]:
# Путь к данным: Colab — монтируем Drive; локально — текущая папка
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_PATH = '/content/drive/MyDrive/llm_filter_data'
except ImportError:
    DATA_PATH = '.'  # локально: train.parquet и test.parquet в текущей папке

## 2. Загрузка и подготовка данных

In [ ]:
print("Загрузка данных...")
train_df = pd.read_parquet(os.path.join(DATA_PATH, 'train.parquet'))
test_df = pd.read_parquet(os.path.join(DATA_PATH, 'test.parquet'))
print(f"Train: {len(train_df)}, Test: {len(test_df)}")
train_df.head()

In [ ]:
# Фильтруем только 0 и 1 (примеры с 0.5 не участвуют в метрике)
train_clean = train_df[
    (train_df['integrity'].isin([0, 1])) &
    (train_df['factuality'].isin([0, 1])) &
    (train_df['truthfulness'].isin([0, 1]))
].copy()

strat_cols = train_clean[['integrity', 'factuality', 'truthfulness']].apply(
    lambda x: x.astype(str).str.cat(), axis=1
)
train_part, val_part = train_test_split(
    train_clean,
    test_size=0.2,
    random_state=42,
    stratify=strat_cols
)
print(f"Train: {len(train_part)}, Val: {len(val_part)}")

## 3. Токенизация

In [ ]:
MODEL_NAME = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=512
    )

def get_remove_columns(dataset):
    return [col for col in dataset.column_names if col != 'text']

# Трейн
train_ds = Dataset.from_pandas(train_part[['text']].copy(), preserve_index=False)
train_tok = train_ds.map(tokenize_fn, batched=True, remove_columns=get_remove_columns(train_ds))
train_tok = train_tok.add_column('integrity', train_part['integrity'].tolist())
train_tok = train_tok.add_column('factuality', train_part['factuality'].tolist())
train_tok = train_tok.add_column('truthfulness', train_part['truthfulness'].tolist())

# Валидация
val_ds = Dataset.from_pandas(val_part[['text']].copy(), preserve_index=False)
val_tok = val_ds.map(tokenize_fn, batched=True, remove_columns=get_remove_columns(val_ds))
val_tok = val_tok.add_column('integrity', val_part['integrity'].tolist())
val_tok = val_tok.add_column('factuality', val_part['factuality'].tolist())
val_tok = val_tok.add_column('truthfulness', val_part['truthfulness'].tolist())

In [ ]:
class MultiTaskDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        return {
            'input_ids': torch.tensor(item['input_ids']),
            'attention_mask': torch.tensor(item['attention_mask']),
            'integrity': torch.tensor(item['integrity']),
            'factuality': torch.tensor(item['factuality']),
            'truthfulness': torch.tensor(item['truthfulness'])
        }

train_multitask = MultiTaskDataset(train_tok)
val_multitask = MultiTaskDataset(val_tok)
print(f"Train: {len(train_multitask)}, Val: {len(val_multitask)}")

## 4. Модель: Multi-Task BERT с кастомным loss

In [ ]:
import torch.nn as nn

class F1Loss(nn.Module):
    def __init__(self, beta=1.0, eps=1e-7):
        super().__init__()
        self.beta = beta
        self.eps = eps

    def forward(self, logits, targets, pos_label=0):
        probs = torch.softmax(logits, dim=1)[:, pos_label]
        binary_targets = (targets == pos_label).float()
        tp = (probs * binary_targets).sum()
        fp = (probs * (1 - binary_targets)).sum()
        fn = ((1 - probs) * binary_targets).sum()
        precision = tp / (tp + fp + self.eps)
        recall = tp / (tp + fn + self.eps)
        f1 = (1 + self.beta**2) * precision * recall / (self.beta**2 * precision + recall + self.eps)
        return 1 - f1

CLASS_WEIGHTS = {
    'integrity': torch.tensor([1.74, 0.26]),
    'factuality': torch.tensor([1.67, 0.33]),
    'truthfulness': torch.tensor([1.45, 0.55])
}

class MultiTaskBERT(nn.Module):
    def __init__(self, model_name, num_labels=2, weights=None, unfreeze=2):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        for param in self.bert.parameters():
            param.requires_grad = False
        for layer in self.bert.encoder.layer[-unfreeze:]:
            for param in layer.parameters():
                param.requires_grad = True
        for param in self.bert.pooler.parameters():
            param.requires_grad = True

        self.weights = weights or {k: torch.tensor([1.0, 1.0]) for k in ['integrity', 'factuality', 'truthfulness']}
        hidden = self.bert.config.hidden_size
        self.head_integrity = nn.Sequential(nn.Dropout(0.3), nn.Linear(hidden, 256), nn.ReLU(), nn.Dropout(0.3), nn.Linear(256, num_labels))
        self.head_factuality = nn.Sequential(nn.Dropout(0.3), nn.Linear(hidden, 256), nn.ReLU(), nn.Dropout(0.3), nn.Linear(256, num_labels))
        self.head_truthfulness = nn.Sequential(nn.Dropout(0.3), nn.Linear(hidden, 256), nn.ReLU(), nn.Dropout(0.3), nn.Linear(256, num_labels))
        self.f1_loss = F1Loss()

    def forward(self, input_ids, attention_mask, integrity=None, factuality=None, truthfulness=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        pooled = outputs.pooler_output
        logits_i = self.head_integrity(pooled)
        logits_f = self.head_factuality(pooled)
        logits_t = self.head_truthfulness(pooled)

        loss = None
        if all(l is not None for l in [integrity, factuality, truthfulness]):
            device = integrity.device
            f1_i = self.f1_loss(logits_i, integrity.long(), pos_label=0)
            f1_f = self.f1_loss(logits_f, factuality.long(), pos_label=0)
            f1_t = self.f1_loss(logits_t, truthfulness.long(), pos_label=0)
            w_i = self.weights['integrity'].to(device)
            w_f = self.weights['factuality'].to(device)
            w_t = self.weights['truthfulness'].to(device)
            ce_i = nn.CrossEntropyLoss(weight=w_i)(logits_i, integrity.long())
            ce_f = nn.CrossEntropyLoss(weight=w_f)(logits_f, factuality.long())
            ce_t = nn.CrossEntropyLoss(weight=w_t)(logits_t, truthfulness.long())
            loss_i = 0.7 * f1_i + 0.3 * ce_i
            loss_f = 0.7 * f1_f + 0.3 * ce_f
            loss_t = 0.7 * f1_t + 0.3 * ce_t
            loss = (loss_i + loss_f + loss_t) / 3

        return {
            'loss': loss,
            'logits_integrity': logits_i,
            'logits_factuality': logits_f,
            'logits_truthfulness': logits_t
        }

## 5. Кастомный Trainer

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels_i = inputs.pop("integrity")
        labels_f = inputs.pop("factuality")
        labels_t = inputs.pop("truthfulness")
        outputs = model(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            integrity=labels_i,
            factuality=labels_f,
            truthfulness=labels_t
        )
        return (outputs['loss'], outputs) if return_outputs else outputs['loss']

    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        dataloader = self.get_eval_dataloader(eval_dataset)
        total_loss, num_batches = 0, 0
        preds_i, labels_i = [], []
        preds_f, labels_f = [], []
        preds_t, labels_t = [], []
        self.model.eval()
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Оценка"):
                batch = {k: v.to(self.args.device) for k, v in batch.items()}
                l_i = batch.pop("integrity")
                l_f = batch.pop("factuality")
                l_t = batch.pop("truthfulness")
                outputs = self.model(**batch, integrity=l_i, factuality=l_f, truthfulness=l_t)
                if outputs['loss'] is not None:
                    total_loss += outputs['loss'].item()
                    num_batches += 1
                preds_i.extend(torch.argmax(outputs['logits_integrity'], dim=1).cpu().numpy())
                labels_i.extend(l_i.cpu().numpy())
                preds_f.extend(torch.argmax(outputs['logits_factuality'], dim=1).cpu().numpy())
                labels_f.extend(l_f.cpu().numpy())
                preds_t.extend(torch.argmax(outputs['logits_truthfulness'], dim=1).cpu().numpy())
                labels_t.extend(l_t.cpu().numpy())
        metrics = {
            f'{metric_key_prefix}_loss': total_loss / num_batches if num_batches else 0,
            f'{metric_key_prefix}_integrity_f1_0': f1_score(labels_i, preds_i, pos_label=0, average='binary'),
            f'{metric_key_prefix}_factuality_f1_0': f1_score(labels_f, preds_f, pos_label=0, average='binary'),
            f'{metric_key_prefix}_truthfulness_f1_0': f1_score(labels_t, preds_t, pos_label=0, average='binary'),
        }
        return metrics

## 6. Обучение

In [ ]:
model = MultiTaskBERT(MODEL_NAME, weights=CLASS_WEIGHTS, unfreeze=2)
print(f"Обучаемых параметров: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

args = TrainingArguments(
    output_dir='./model_output',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=1e-5,
    weight_decay=0.1,
    logging_steps=50,
    eval_strategy='epoch',
    save_strategy='no',
    fp16=torch.cuda.is_available(),
    disable_tqdm=False,
    report_to='none',
    dataloader_num_workers=2
)

trainer = CustomTrainer(
    model=model,
    args=args,
    train_dataset=train_multitask,
    eval_dataset=val_multitask
)
trainer.train()

## 7. Оценка на валидации

In [ ]:
device = next(model.parameters()).device
val_loader = torch.utils.data.DataLoader(val_multitask, batch_size=16, shuffle=False)
all_preds = {'integrity': [], 'factuality': [], 'truthfulness': []}
all_labels = {'integrity': [], 'factuality': [], 'truthfulness': []}

model.eval()
with torch.no_grad():
    for batch in tqdm(val_loader, desc="Валидация"):
        batch = {k: v.to(device) for k, v in batch.items()}
        labels_i = batch.pop('integrity')
        labels_f = batch.pop('factuality')
        labels_t = batch.pop('truthfulness')
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        all_preds['integrity'].extend(torch.argmax(outputs['logits_integrity'], dim=1).cpu().numpy())
        all_preds['factuality'].extend(torch.argmax(outputs['logits_factuality'], dim=1).cpu().numpy())
        all_preds['truthfulness'].extend(torch.argmax(outputs['logits_truthfulness'], dim=1).cpu().numpy())
        all_labels['integrity'].extend(labels_i.cpu().numpy())
        all_labels['factuality'].extend(labels_f.cpu().numpy())
        all_labels['truthfulness'].extend(labels_t.cpu().numpy())

f1_scores = []
for task in ['integrity', 'factuality', 'truthfulness']:
    f1_0 = f1_score(all_labels[task], all_preds[task], pos_label=0, average='binary')
    f1_1 = f1_score(all_labels[task], all_preds[task], pos_label=1, average='binary')
    f1_scores.append(f1_0)
    print(f"{task}: F1@0={f1_0:.4f}, F1@1={f1_1:.4f}")
print(f"\nСредний F1 (по классу 0): {np.mean(f1_scores):.4f}")

## 8. Предсказание на тесте и формирование submission

In [ ]:
test_ds = Dataset.from_pandas(test_df[['text']].copy(), preserve_index=False)
remove_cols = [c for c in test_ds.column_names if c != 'text']
test_tok = test_ds.map(tokenize_fn, batched=True, remove_columns=remove_cols)

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        item = self.dataset[idx]
        return {'input_ids': torch.tensor(item['input_ids']), 'attention_mask': torch.tensor(item['attention_mask'])}

test_dataset = TestDataset(test_tok)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

model.eval()
preds_integrity, preds_factuality, preds_truthfulness = [], [], []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Инференс"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        preds_integrity.extend(torch.argmax(outputs['logits_integrity'], dim=1).cpu().numpy())
        preds_factuality.extend(torch.argmax(outputs['logits_factuality'], dim=1).cpu().numpy())
        preds_truthfulness.extend(torch.argmax(outputs['logits_truthfulness'], dim=1).cpu().numpy())

id_col = 'uuid' if 'uuid' in test_df.columns else ('id' if 'id' in test_df.columns else None)
uuid_values = test_df[id_col].astype(str) if id_col else test_df.index.astype(str)

submission = pd.DataFrame({
    'uuid': uuid_values,
    'integrity': preds_integrity,
    'truthfulness': preds_truthfulness,
    'factuality': preds_factuality
})

submission_path = os.path.join(DATA_PATH, 'submission.csv')
submission.to_csv(submission_path, index=False)
print(f"Submission сохранён: {submission_path}")
submission.head()